# Исследование надежности заемщиков


## Загрузка данных и получение информации о данных.

In [714]:
import pandas as pd
from google.colab import data_table

data_table.enable_dataframe_formatter()

try:
  df = pd.read_csv('drive/MyDrive/datasets/data.csv')
except:
  df = pd.read_csv('https://code.s3.yandex.net/datasets/data.csv')

## Обзор полученных данных



In [715]:
def info_df(data):
  display(data.head(10))
  display(data.info())
  print('')
  print('Количество пропусков в процентах')
  display(pd.DataFrame(round(data.isna().mean()*100, 2)).style.background_gradient())

display(df.head(10))
display(df.info())
df.describe()

# info_df(df)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


None

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


Выводы по таблице.
Перед нами данные клентов банка. В ней представленны такие данные как доход, семейное положение, образование и т. д. На основании этих данны попробуем провести анализ способности потенциального заемщика вернуть кредит банку. В представленных данных есть проблемы, которые небходимо решить: пропуски данных, странные данные в стаже клиента, разный регистр в образовании.

## Рассмотрим каждый столбец по отдельности. 


### Данные о детях

In [716]:
df['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Количество детей 20 и -1 выглядит странно. Скорей всего это опечатка и тут следует подставить 2 и 1 соответственно.

In [717]:
df['children'] = df['children'].replace(20, 2)
df['children'] = df['children'].replace(-1, 1)
df['children'].unique()

array([1, 0, 3, 2, 4, 5])

Приведем данные об оброзовании к единому регистру.

In [718]:
df['education'] = df['education'].str.lower()
df['education'].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

### Семейное положение
Тут все в порядке

In [719]:
df['family_status'].value_counts()


женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

### Рассмотрим статус клиента
Все ок.

In [720]:
df['income_type'].value_counts()

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
безработный            2
предприниматель        2
студент                1
в декрете              1
Name: income_type, dtype: int64

### Рассмотрим столбец пол

In [721]:
df['gender'].value_counts()
df.loc[df['gender'] == 'XNA']

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
10701,0,-2358.600502,24,неоконченное высшее,2,гражданский брак,1,XNA,компаньон,0,203905.157261,покупка недвижимости


Странные данные в одном столбце. Удалим его.

In [722]:
df.drop(df.index[10701], inplace=True)

print(df['gender'].unique())
df.reset_index(drop=False)
print(df.index[10701])

['F' 'M']
10702


### Стаж клиента 
Стаж указан в днях. Некоторые данные с отрицательными значениями, их на до заменить на положительные. Некоторые значения некорректы, например стаж больше возраста - здесь надо подумать что сделать. Для наглядности стаж лучше перевестим в года.

In [723]:
df['days_employed'].describe()

count     19350.000000
mean      63049.877770
std      140830.166081
min      -18388.949901
25%       -2747.486276
50%       -1203.307346
75%        -291.077541
max      401755.400475
Name: days_employed, dtype: float64

Заменим отрицательные данные на положительные.

In [724]:
df['days_employed'] = df['days_employed'].abs()

display(df['days_employed'].head(10))

0      8437.673028
1      4024.803754
2      5623.422610
3      4124.747207
4    340266.072047
5       926.185831
6      2879.202052
7       152.779569
8      6929.865299
9      2188.756445
Name: days_employed, dtype: float64

Данные свыше 20000 выглядят странно т.к. в годах это более 55 лет, что бывает крайне редко, а более 100 не встречается никогда. Скорей всего при выгрузке данных произошла ошибка и плавующая точка сметистилась на две позиции. Так же переведем дни в года и отбросим дробную часть, таблицу переименуем. Попробуем восстановить данные.

In [725]:
df.loc[(df['days_employed'] > 20000), 'days_employed'] = df.loc[(df['days_employed'] > 25000), 'days_employed'] / 100


df['days_employed'] = df['days_employed'].fillna(0)
df['days_employed'] = df['days_employed'] / 365
df['days_employed'] = df['days_employed'].astype(int)

print(df['days_employed'].describe())

display(df.head(10))

count    21524.000000
mean         5.915072
std          5.931052
min          0.000000
25%          1.000000
50%          4.000000
75%          9.000000
max         50.000000
Name: days_employed, dtype: float64


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,23,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,11,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,15,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,11,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,9,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,2,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,7,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,0,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,18,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,5,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


### Доход клиента
Рассмотрим доход


In [726]:
df.loc[df['education'].str.lower()=='высшее', 'total_income']


0        253875.639453
5        255763.565419
6        240525.971920
8         95856.832424
10       113943.491460
             ...      
21509     73029.059379
21511    140068.472941
21512    147301.457769
21516    322807.776603
21517    178059.553491
Name: total_income, Length: 5260, dtype: float64

В столбце дохода (total_income) имеются пропуски. Имеет смысл заменить эти пропуски для тех заемщиков, которые работают медианным значением.

In [727]:
df.loc[(df['total_income'].isna())&(df['income_type'].str.lower() == 'сотрудник'),'total_income']\
  = df.loc[df['income_type'].str.lower() == 'сотрудник','total_income'].median()

info_df(df)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,23,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,11,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,15,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,11,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,9,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,2,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,7,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,0,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,18,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,5,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


<class 'pandas.core.frame.DataFrame'>
Int64Index: 21524 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21524 non-null  int64  
 1   days_employed     21524 non-null  int64  
 2   dob_years         21524 non-null  int64  
 3   education         21524 non-null  object 
 4   education_id      21524 non-null  int64  
 5   family_status     21524 non-null  object 
 6   family_status_id  21524 non-null  int64  
 7   gender            21524 non-null  object 
 8   income_type       21524 non-null  object 
 9   debt              21524 non-null  int64  
 10  total_income      20455 non-null  float64
 11  purpose           21524 non-null  object 
dtypes: float64(1), int64(6), object(5)
memory usage: 2.1+ MB


None


Количество пропусков в процентах


,0
children,0.000000
days_employed,0.000000
dob_years,0.000000
education,0.000000
education_id,0.000000
family_status,0.000000
family_status_id,0.000000
gender,0.000000
income_type,0.000000
debt,0.000000
